In [3]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from collections import Counter

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold
import lightgbm as lgb
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
##### mode(top) + other 2/3 features
#### holiday
#### aggregation by time features

In [94]:
#### read files
df_train = pd.read_csv("/Users/s0c02nj/Desktop/WNS2/train_NA17Sgz/train.csv")
df_log = pd.read_csv("/Users/s0c02nj/Desktop/WNS2/train_NA17Sgz/view_log.csv")
df_item = pd.read_csv("/Users/s0c02nj/Desktop/WNS2/train_NA17Sgz/item_data.csv")

In [96]:
df_test = pd.read_csv("/Users/s0c02nj/Desktop/WNS2/test.csv")

In [97]:
df_train.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0


In [98]:
df_log.head()

,server_time,device_type,session_id,user_id,item_id
0,2018-10-15 08:58:00,android,112333,4557,32970
1,2018-10-15 08:58:00,android,503590,74788,7640
2,2018-10-15 08:58:00,android,573960,23628,128855
3,2018-10-15 08:58:00,android,121691,2430,12774
4,2018-10-15 08:58:00,android,218564,19227,28296


In [99]:
df_item.head()

,item_id,item_price,category_1,category_2,category_3,product_type
0,26880,4602,11,35,20,3040
1,54939,3513,12,57,85,6822
2,40383,825,17,8,279,1619
3,8777,2355,13,58,189,5264
4,113705,1267,17,39,151,10239


In [100]:
df_log_item = pd.merge(left = df_log, 
                  right= df_item, 
                  how='left',
                  left_on=['item_id'], 
                  right_on=['item_id'])

In [101]:
df_log_item.head()

,server_time,device_type,session_id,user_id,item_id,item_price,category_1,category_2,category_3,product_type
0,2018-10-15 08:58:00,android,112333,4557,32970,54685.0,16.0,56.0,253.0,3184.0
1,2018-10-15 08:58:00,android,503590,74788,7640,1376.0,7.0,71.0,228.0,545.0
2,2018-10-15 08:58:00,android,573960,23628,128855,4544.0,4.0,38.0,62.0,5609.0
3,2018-10-15 08:58:00,android,121691,2430,12774,904.0,17.0,39.0,252.0,2740.0
4,2018-10-15 08:58:00,android,218564,19227,28296,2304.0,12.0,57.0,54.0,7422.0


In [102]:
cat_cols = ['device_type', 'category_1', 'category_2', 'category_3', 'product_type', 'item_id']

for col in cat_cols:
    le = preprocessing.LabelEncoder()
    df_log_item[col] = le.fit_transform(df_log_item[col])

In [103]:
df_log_item["cat1_cat2"] = df_log_item["category_1"].astype("str") + df_log_item["category_2"].astype("str")
df_log_item["cat1_cat3"] = df_log_item["category_1"].astype("str") + df_log_item["category_3"].astype("str")
df_log_item["cat2_cat3"] = df_log_item["category_2"].astype("str") + df_log_item["category_3"].astype("str")
df_log_item["cat1_cat2_cat3"] = df_log_item["category_1"].astype("str") + df_log_item["category_2"].astype("str") + df_log_item["category_3"].astype("str")



In [104]:
##### server feature week, month, day and hour
df_log_item['server_time'] = pd.to_datetime(df_log_item['server_time'])

df_log_item['server_wk'] = df_log_item['server_time'].dt.week
df_log_item['server_mth'] = df_log_item['server_time'].dt.month
df_log_item['server_day'] = df_log_item['server_time'].dt.day
df_log_item['server_hr'] = df_log_item['server_time'].dt.hour
df_log_item['server_weekend'] = (df_log_item['server_time'].dt.weekday >= 5).astype(int)

In [105]:
### impute price
price_med = df_log_item['item_price'].median()
df_log_item['item_price'] = df_log_item['item_price'].replace(np.nan, price_med, regex = True)

In [106]:
df_log_item['item_price'] = df_log_item['item_price'].apply(lambda x:np.log1p(x))

In [107]:
def check_nas(df):
    print(df.isna().sum())

In [108]:
def q3(x):
    return np.percentile(x , 75)

def q1(x):
    return np.percentile(x , 25)

def mode(x):
    return Counter(x).most_common(1)[0][0]

def mode_1(x):
    try:
        return Counter(x).most_common(2)[1][0]
    except:
        return 999999

def mode_2(x):    
    try:
        return Counter(x).most_common(3)[2][0]
    except:
        return 999999

In [109]:
x = [1,1,1,1,1,1,1,2,2,2,2,3]

In [110]:
mode_1(x)

2

In [111]:
is_click = df_train["is_click"]
df_train = df_train.drop(["is_click"], axis=1)

In [112]:
df_combined = pd.concat([df_train, df_test])

In [24]:
aggs = {}
aggs['item_price'] = ['median', 'mean', 'sum', 'max', 'min', q1, q3,  mode, mode_1, mode_2]
aggs['device_type'] = ['count', 'nunique', mode, mode_1, mode_2]
aggs['user_id'] = ['count', 'nunique']
aggs['session_id'] = ['count', 'nunique']
aggs['item_id'] = ['nunique', mode, mode_1, mode_2]
aggs['category_1'] = ['nunique', mode, mode_1, mode_2]
aggs['category_2'] = ['nunique', mode, mode_1, mode_2]
aggs['category_3'] = ['nunique', mode, mode_1, mode_2]
aggs['cat1_cat2'] = ['nunique', mode, mode_1, mode_2]
aggs['cat1_cat3'] = ['nunique', mode, mode_1, mode_2]
aggs['cat2_cat3'] = ['nunique', mode, mode_1, mode_2]
aggs['cat1_cat2_cat3'] = ['nunique', mode, mode_1, mode_2]

aggs['server_wk'] = ['nunique', mode, mode_1, mode_2, 'mean']
aggs['server_mth'] = ['nunique', mode, mode_1, mode_2, 'mean']
aggs['server_day'] = ['nunique', mode, mode_1, mode_2, 'mean']
aggs['server_hr'] = ['nunique', mode, mode_1, mode_2, 'mean']
aggs['server_weekend'] = ['nunique', mode, mode_1, mode_2, 'sum']

In [25]:
for col in tqdm(["server_day", "server_hr"]):
    
    aggs_temp = aggs.copy()
    aggs_temp.pop(col)
    
    agg_df = df_log_item.groupby(col).agg(aggs_temp).reset_index()
    agg_df.columns = [col] + [col + '_' + c[0] +'_' + c[1] for c in agg_df.columns[1:]]
    
    df_log_item = pd.merge(left = df_log_item, right= agg_df, how='left',
                    left_on=[col], right_on=[col])

100%|██████████| 2/2 [02:17<00:00, 64.00s/it]


In [26]:
for col in tqdm(["user_id"]):
    
    aggs_temp = aggs.copy()
    aggs_temp.pop(col)
    
    agg_df = df_log_item.groupby(col).agg(aggs_temp).reset_index()
    agg_df.columns = [col] + [col + '_' + c[0] +'_' + c[1] for c in agg_df.columns[1:]]
    
    df_final = pd.merge(left = df_combined, right= agg_df, how='left',
                    left_on=[col], right_on=[col])

100%|██████████| 1/1 [03:21<00:00, 201.55s/it]


In [27]:
df_train.shape[0] == df_final.shape[0]

False

In [28]:
df_final.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,user_id_item_price_median,user_id_item_price_mean,user_id_item_price_sum,user_id_item_price_max,...,user_id_server_hr_nunique,user_id_server_hr_mode,user_id_server_hr_mode_1,user_id_server_hr_mode_2,user_id_server_hr_mean,user_id_server_weekend_nunique,user_id_server_weekend_mode,user_id_server_weekend_mode_1,user_id_server_weekend_mode_2,user_id_server_weekend_sum
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,7.762596,7.762596,7.762596,7.762596,...,1,18,999999,999999,18.000000,1,0,999999,999999,0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,8.137980,8.199240,98.390874,9.558671,...,7,5,19,6,11.750000,2,0,1,999999,4
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,7.294462,7.294462,14.588923,7.707512,...,1,12,999999,999999,12.000000,1,0,999999,999999,0
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,7.695744,7.971887,143.493966,11.446454,...,7,18,12,19,16.444444,2,0,1,999999,5
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,8.187685,8.094990,372.369547,11.673146,...,13,22,23,5,16.065217,2,0,1,999999,5


In [29]:
df_final['impression_time'] = pd.to_datetime(df_final['impression_time'])

df_final['impression_wk'] = df_final['impression_time'].dt.week
df_final['impression_mth'] = df_final['impression_time'].dt.month
df_final['impression_day'] = df_final['impression_time'].dt.day
df_final['impression_hr'] = df_final['impression_time'].dt.hour

df_final['impression_weekend'] = (df_final['impression_time'].dt.weekday >= 5).astype(int)

In [31]:
df_final = df_final.drop(['impression_id', 'impression_time', 'user_id'], axis =1)

In [32]:
col_counts = []

for col in tqdm(df_final.columns):
    
    counter = Counter(df_final[col])
    col_counts.append(str(col)+'_count')
    
    df_final[str(col)+'_count'] = df_final[col].apply(lambda x:counter[x])

100%|██████████| 82/82 [00:13<00:00,  7.43it/s]


In [33]:
df_final.shape

(328284, 164)

In [34]:
df_final.head()

,app_code,os_version,is_4G,user_id_item_price_median,user_id_item_price_mean,user_id_item_price_sum,user_id_item_price_max,user_id_item_price_min,user_id_item_price_q1,user_id_item_price_q3,...,user_id_server_weekend_nunique_count,user_id_server_weekend_mode_count,user_id_server_weekend_mode_1_count,user_id_server_weekend_mode_2_count,user_id_server_weekend_sum_count,impression_wk_count,impression_mth_count,impression_day_count,impression_hr_count,impression_weekend_count
0,422,old,0,7.762596,7.762596,7.762596,7.762596,7.762596,7.762596,7.762596,...,69589,269441,69589,328284,57584,28702,131317,24078,28834,237139
1,467,latest,1,8.137980,8.199240,98.390874,9.558671,6.932448,8.072265,8.401682,...,258695,269441,211857,328284,15867,28702,131317,24078,28834,237139
2,259,intermediate,1,7.294462,7.294462,14.588923,7.707512,6.881411,7.087937,7.500987,...,69589,269441,69589,328284,57584,28702,131317,24078,28834,237139
3,244,latest,1,7.695744,7.971887,143.493966,11.446454,5.521461,7.379008,8.898502,...,258695,269441,211857,328284,14373,28702,131317,24078,28834,237139
4,473,latest,0,8.187685,8.094990,372.369547,11.673146,5.666427,7.084755,8.530450,...,258695,269441,211857,328284,14373,28702,131317,24078,28834,237139


In [35]:
cat_appended_cols = []
for col in df_final.columns:
    if df_final[col].dtype == "object":
        cat_appended_cols.append(col)
        print(col , df_final[col].dtype)

os_version object
user_id_cat1_cat2_mode object
user_id_cat1_cat2_mode_1 object
user_id_cat1_cat2_mode_2 object
user_id_cat1_cat3_mode object
user_id_cat1_cat3_mode_1 object
user_id_cat1_cat3_mode_2 object
user_id_cat2_cat3_mode object
user_id_cat2_cat3_mode_1 object
user_id_cat2_cat3_mode_2 object
user_id_cat1_cat2_cat3_mode object
user_id_cat1_cat2_cat3_mode_1 object
user_id_cat1_cat2_cat3_mode_2 object


In [38]:
cat_cols_train = ['app_code', 'os_version'] + cat_appended_cols

for col in cat_cols_train:
    print(col)
    le = preprocessing.LabelEncoder()
    df_final[col] = le.fit_transform(df_final[col].astype(int))

app_code
os_version
os_version
user_id_cat1_cat2_mode
user_id_cat1_cat2_mode_1
user_id_cat1_cat2_mode_2
user_id_cat1_cat3_mode
user_id_cat1_cat3_mode_1
user_id_cat1_cat3_mode_2
user_id_cat2_cat3_mode
user_id_cat2_cat3_mode_1
user_id_cat2_cat3_mode_2
user_id_cat1_cat2_cat3_mode
user_id_cat1_cat2_cat3_mode_1
user_id_cat1_cat2_cat3_mode_2


In [115]:
df_final1['y'] = y


In [83]:
x_train = df_final[0 : df_train.shape[0]]
x_test = df_final[df_train.shape[0] : ]
y = is_click

In [84]:
x_t, x_v, y_t, y_v = train_test_split(x_train, y, stratify=y, random_state=42, test_size=0.2)

In [87]:
params = {
    'boost': 'gbdt',
    'learning_rate': 0.03,
    'max_depth': 25,  
    'metric':{'auc'},
    'num_threads': -1,
    'objective': 'binary', 
    'verbosity': 1,
    'num_leaves': 10,
    'bagging_fraction':0.6, #rows
    'bagging_frequency':20,
    'min_data_in_leaf':3,
    'feature_fraction':0.4, # features
    'lambda_l1':5,
    'lambda_l2':10,
    'min_gain_split':0.4,
    'is_unbalance':True
}

In [88]:
d_train = lgb.Dataset(x_t, label=y_t)
d_val = lgb.Dataset(x_v, label=y_v)

clf = lgb.train(params, d_train, 150000, verbose_eval=1, 
                valid_sets = [d_train, d_val], early_stopping_rounds = 500)

[1]	training's auc: 0.673888	valid_1's auc: 0.663076
Training until validation scores don't improve for 500 rounds.
[2]	training's auc: 0.694598	valid_1's auc: 0.684194
[3]	training's auc: 0.696074	valid_1's auc: 0.686009
[4]	training's auc: 0.70029	valid_1's auc: 0.689015
[5]	training's auc: 0.702331	valid_1's auc: 0.690404
[6]	training's auc: 0.704595	valid_1's auc: 0.692127
[7]	training's auc: 0.70725	valid_1's auc: 0.694155
[8]	training's auc: 0.707641	valid_1's auc: 0.694685
[9]	training's auc: 0.707927	valid_1's auc: 0.694648
[10]	training's auc: 0.709409	valid_1's auc: 0.696034
[11]	training's auc: 0.710525	valid_1's auc: 0.69726
[12]	training's auc: 0.711179	valid_1's auc: 0.697536
[13]	training's auc: 0.712991	valid_1's auc: 0.699192
[14]	training's auc: 0.713837	valid_1's auc: 0.699651
[15]	training's auc: 0.714535	valid_1's auc: 0.700586
[16]	training's auc: 0.715185	valid_1's auc: 0.700953
[17]	training's auc: 0.716127	valid_1's auc: 0.701545
[18]	training's auc: 0.717566	v

[154]	training's auc: 0.746986	valid_1's auc: 0.725725
[155]	training's auc: 0.747094	valid_1's auc: 0.72582
[156]	training's auc: 0.747258	valid_1's auc: 0.725858
[157]	training's auc: 0.747466	valid_1's auc: 0.725943
[158]	training's auc: 0.747682	valid_1's auc: 0.726
[159]	training's auc: 0.747753	valid_1's auc: 0.726001
[160]	training's auc: 0.747864	valid_1's auc: 0.726034
[161]	training's auc: 0.748081	valid_1's auc: 0.726234
[162]	training's auc: 0.748269	valid_1's auc: 0.726301
[163]	training's auc: 0.748467	valid_1's auc: 0.726307
[164]	training's auc: 0.748698	valid_1's auc: 0.72654
[165]	training's auc: 0.748917	valid_1's auc: 0.726677
[166]	training's auc: 0.74899	valid_1's auc: 0.726685
[167]	training's auc: 0.749169	valid_1's auc: 0.726843
[168]	training's auc: 0.749246	valid_1's auc: 0.726843
[169]	training's auc: 0.749329	valid_1's auc: 0.726956
[170]	training's auc: 0.749455	valid_1's auc: 0.727011
[171]	training's auc: 0.749617	valid_1's auc: 0.727071
[172]	training's

[307]	training's auc: 0.767487	valid_1's auc: 0.733866
[308]	training's auc: 0.767575	valid_1's auc: 0.733915
[309]	training's auc: 0.76774	valid_1's auc: 0.733937
[310]	training's auc: 0.767843	valid_1's auc: 0.733968
[311]	training's auc: 0.768051	valid_1's auc: 0.734063
[312]	training's auc: 0.76816	valid_1's auc: 0.734118
[313]	training's auc: 0.768324	valid_1's auc: 0.734268
[314]	training's auc: 0.768458	valid_1's auc: 0.734313
[315]	training's auc: 0.768629	valid_1's auc: 0.734412
[316]	training's auc: 0.768727	valid_1's auc: 0.734427
[317]	training's auc: 0.768837	valid_1's auc: 0.734506
[318]	training's auc: 0.768947	valid_1's auc: 0.734548
[319]	training's auc: 0.769034	valid_1's auc: 0.734602
[320]	training's auc: 0.769153	valid_1's auc: 0.73463
[321]	training's auc: 0.769224	valid_1's auc: 0.734665
[322]	training's auc: 0.769349	valid_1's auc: 0.734665
[323]	training's auc: 0.7694	valid_1's auc: 0.734649
[324]	training's auc: 0.769498	valid_1's auc: 0.734705
[325]	training'

[463]	training's auc: 0.784016	valid_1's auc: 0.738211
[464]	training's auc: 0.784119	valid_1's auc: 0.738243
[465]	training's auc: 0.7842	valid_1's auc: 0.738269
[466]	training's auc: 0.784333	valid_1's auc: 0.73831
[467]	training's auc: 0.784406	valid_1's auc: 0.738332
[468]	training's auc: 0.784471	valid_1's auc: 0.738357
[469]	training's auc: 0.784578	valid_1's auc: 0.738411
[470]	training's auc: 0.784671	valid_1's auc: 0.738418
[471]	training's auc: 0.784741	valid_1's auc: 0.738422
[472]	training's auc: 0.784839	valid_1's auc: 0.738464
[473]	training's auc: 0.784958	valid_1's auc: 0.738493
[474]	training's auc: 0.785069	valid_1's auc: 0.738553
[475]	training's auc: 0.785154	valid_1's auc: 0.738624
[476]	training's auc: 0.785218	valid_1's auc: 0.73873
[477]	training's auc: 0.785318	valid_1's auc: 0.738709
[478]	training's auc: 0.78539	valid_1's auc: 0.738674
[479]	training's auc: 0.785513	valid_1's auc: 0.738713
[480]	training's auc: 0.785626	valid_1's auc: 0.738741
[481]	training'

[618]	training's auc: 0.797582	valid_1's auc: 0.74074
[619]	training's auc: 0.797666	valid_1's auc: 0.74079
[620]	training's auc: 0.797761	valid_1's auc: 0.740828
[621]	training's auc: 0.797836	valid_1's auc: 0.740863
[622]	training's auc: 0.797907	valid_1's auc: 0.740858
[623]	training's auc: 0.797982	valid_1's auc: 0.740881
[624]	training's auc: 0.798038	valid_1's auc: 0.740884
[625]	training's auc: 0.798149	valid_1's auc: 0.740864
[626]	training's auc: 0.798234	valid_1's auc: 0.740848
[627]	training's auc: 0.798338	valid_1's auc: 0.740852
[628]	training's auc: 0.798342	valid_1's auc: 0.740834
[629]	training's auc: 0.798412	valid_1's auc: 0.740861
[630]	training's auc: 0.798479	valid_1's auc: 0.740862
[631]	training's auc: 0.79858	valid_1's auc: 0.740856
[632]	training's auc: 0.798686	valid_1's auc: 0.740863
[633]	training's auc: 0.798753	valid_1's auc: 0.74087
[634]	training's auc: 0.79883	valid_1's auc: 0.740854
[635]	training's auc: 0.798901	valid_1's auc: 0.740861
[636]	training'

...

[2137]	training's auc: 0.876368	valid_1's auc: 0.748549
[2138]	training's auc: 0.876408	valid_1's auc: 0.748551
[2139]	training's auc: 0.876439	valid_1's auc: 0.748563
[2140]	training's auc: 0.876471	valid_1's auc: 0.748576
[2141]	training's auc: 0.876512	valid_1's auc: 0.748617
[2142]	training's auc: 0.876543	valid_1's auc: 0.748589
[2143]	training's auc: 0.876553	valid_1's auc: 0.748599
[2144]	training's auc: 0.876591	valid_1's auc: 0.748594
[2145]	training's auc: 0.876632	valid_1's auc: 0.748594
[2146]	training's auc: 0.876658	valid_1's auc: 0.748592
[2147]	training's auc: 0.876712	valid_1's auc: 0.748628
[2148]	training's auc: 0.876745	valid_1's auc: 0.748635
[2149]	training's auc: 0.876761	valid_1's auc: 0.74864
[2150]	training's auc: 0.876789	valid_1's auc: 0.748625
[2151]	training's auc: 0.876831	valid_1's auc: 0.748603
[2152]	training's auc: 0.876848	valid_1's auc: 0.74861
[2153]	training's auc: 0.876889	valid_1's auc: 0.748628
[2154]	training's auc: 0.876925	valid_1's auc: 0.7

[2288]	training's auc: 0.881545	valid_1's auc: 0.748457
[2289]	training's auc: 0.88159	valid_1's auc: 0.748475
[2290]	training's auc: 0.88162	valid_1's auc: 0.748481
[2291]	training's auc: 0.881646	valid_1's auc: 0.748475
[2292]	training's auc: 0.88168	valid_1's auc: 0.748458
[2293]	training's auc: 0.881703	valid_1's auc: 0.748454
[2294]	training's auc: 0.881742	valid_1's auc: 0.748455
[2295]	training's auc: 0.881774	valid_1's auc: 0.748444
[2296]	training's auc: 0.881805	valid_1's auc: 0.748434
[2297]	training's auc: 0.881838	valid_1's auc: 0.748432
[2298]	training's auc: 0.881882	valid_1's auc: 0.748459
[2299]	training's auc: 0.881917	valid_1's auc: 0.748446
[2300]	training's auc: 0.881948	valid_1's auc: 0.748438
[2301]	training's auc: 0.881962	valid_1's auc: 0.748433
[2302]	training's auc: 0.88201	valid_1's auc: 0.748417
[2303]	training's auc: 0.882033	valid_1's auc: 0.748398
[2304]	training's auc: 0.88205	valid_1's auc: 0.748399
[2305]	training's auc: 0.882076	valid_1's auc: 0.7484

[2443]	training's auc: 0.886538	valid_1's auc: 0.748597
[2444]	training's auc: 0.886573	valid_1's auc: 0.748589
[2445]	training's auc: 0.886605	valid_1's auc: 0.748583
[2446]	training's auc: 0.88663	valid_1's auc: 0.748594
[2447]	training's auc: 0.886669	valid_1's auc: 0.748596
[2448]	training's auc: 0.88669	valid_1's auc: 0.748614
[2449]	training's auc: 0.88673	valid_1's auc: 0.748616
[2450]	training's auc: 0.886771	valid_1's auc: 0.748615
[2451]	training's auc: 0.886813	valid_1's auc: 0.74864
[2452]	training's auc: 0.886837	valid_1's auc: 0.748634
[2453]	training's auc: 0.886866	valid_1's auc: 0.748597
[2454]	training's auc: 0.886901	valid_1's auc: 0.748603
[2455]	training's auc: 0.886913	valid_1's auc: 0.748591
[2456]	training's auc: 0.886956	valid_1's auc: 0.74856
[2457]	training's auc: 0.887	valid_1's auc: 0.748594
[2458]	training's auc: 0.887023	valid_1's auc: 0.748592
[2459]	training's auc: 0.887058	valid_1's auc: 0.748593
[2460]	training's auc: 0.887075	valid_1's auc: 0.748599


[2600]	training's auc: 0.891322	valid_1's auc: 0.74814
[2601]	training's auc: 0.891358	valid_1's auc: 0.748074
[2602]	training's auc: 0.891393	valid_1's auc: 0.74809
[2603]	training's auc: 0.891432	valid_1's auc: 0.748071
[2604]	training's auc: 0.89146	valid_1's auc: 0.748057
[2605]	training's auc: 0.891477	valid_1's auc: 0.748063
[2606]	training's auc: 0.891515	valid_1's auc: 0.748061
[2607]	training's auc: 0.891549	valid_1's auc: 0.748051
[2608]	training's auc: 0.891594	valid_1's auc: 0.74804
[2609]	training's auc: 0.891632	valid_1's auc: 0.748062
[2610]	training's auc: 0.891667	valid_1's auc: 0.748059
[2611]	training's auc: 0.891696	valid_1's auc: 0.748046
[2612]	training's auc: 0.891722	valid_1's auc: 0.748046
[2613]	training's auc: 0.891744	valid_1's auc: 0.748062
[2614]	training's auc: 0.891785	valid_1's auc: 0.748046
[2615]	training's auc: 0.891803	valid_1's auc: 0.748043
[2616]	training's auc: 0.891843	valid_1's auc: 0.748037
[2617]	training's auc: 0.891848	valid_1's auc: 0.748

[2748]	training's auc: 0.895798	valid_1's auc: 0.7476
[2749]	training's auc: 0.89581	valid_1's auc: 0.747613
[2750]	training's auc: 0.89586	valid_1's auc: 0.747669
[2751]	training's auc: 0.895889	valid_1's auc: 0.747678
[2752]	training's auc: 0.895919	valid_1's auc: 0.747683
[2753]	training's auc: 0.895946	valid_1's auc: 0.747683
[2754]	training's auc: 0.895968	valid_1's auc: 0.747676
[2755]	training's auc: 0.895989	valid_1's auc: 0.747668
[2756]	training's auc: 0.896014	valid_1's auc: 0.747674
[2757]	training's auc: 0.896035	valid_1's auc: 0.747664
[2758]	training's auc: 0.896059	valid_1's auc: 0.747674
[2759]	training's auc: 0.896084	valid_1's auc: 0.747694
[2760]	training's auc: 0.896114	valid_1's auc: 0.747679
[2761]	training's auc: 0.896144	valid_1's auc: 0.747675
[2762]	training's auc: 0.89617	valid_1's auc: 0.747663
[2763]	training's auc: 0.896209	valid_1's auc: 0.747659
[2764]	training's auc: 0.896242	valid_1's auc: 0.747659
[2765]	training's auc: 0.896241	valid_1's auc: 0.7476

In [73]:
predictions = clf.predict(x_test)

In [77]:
sub_df = pd.read_csv('/Users/s0c02nj/Desktop/WNS2/sample_submission.csv')
sub_df['is_click'] = predictions
sub_df.to_csv('/Users/s0c02nj/Desktop/WNS2/output_submission/with_lag.csv', index = False)

In [74]:
feature_imp = pd.DataFrame(sorted(zip(clf.feature_importance(), 
                                      x_train.columns), reverse=True), columns=['Value','Feature'])

In [89]:
#feature_imp

In [ ]:
new_feat_imp = feature_imp[feature_imp['Value'] >84]

In [ ]:
list_new_feat = list(new_feat_imp['Feature'])

In [ ]:
#Stratified K-fold

In [ ]:
kf = StratifiedKFold(n_splits=10, random_state=2019, shuffle=True)

In [ ]:
predictions = np.zeros(len(x_test))
num_models = 0

for train_index, valid_index in tqdm(kf.split(x_train, y)):
    
    d_train = lgb.Dataset(x_train.iloc[train_index], label=y[train_index])
    d_val = lgb.Dataset(x_train.iloc[valid_index], label=y[valid_index])

    clf = lgb.train(params, d_train, 150000, verbose_eval=1000, 
                    valid_sets = [d_train, d_val], early_stopping_rounds = 500)
    

    
    num_models += 1
    
    predictions += clf.predict(x_test)

In [ ]:
predictions_10 = predictions / 10.0

In [ ]:
sub_df = pd.read_csv('sample_submission.csv')
sub_df['is_click'] = predictions_10
sub_df.to_csv('sub_agg_time_and_timeagg_count_top3_8654_75389_lr01_10fold.csv', index = False)

In [ ]:
pred2= (predictions_5 + predictions_10) / 2.0

In [ ]:
sub_df = pd.read_csv('sample_submission.csv')
sub_df['is_click'] = pred2
sub_df.to_csv('sub_agg_time_and_timeagg_count_top3_8654_75389_lr01_10_5_div2fold.csv', index = False)

In [ ]:
temp = df_log_item[df_log_item.user_id == 4557].groupby(['user_id', 'session_id'])['item_id'].apply(list)

In [ ]:
a = []
for user_id in tqdm(list(df_log_item["user_id"].unique())):
    temp = df_log_item[df_log_item.user_id == user_id].groupby(['user_id', 'session_id'])['item_id'].apply(list)
    a += list(temp)

In [ ]:
session_length = [len(t) for t in tqdm(a)]

In [ ]:
np.percentile(session_length, 75)

In [ ]:
Counter(session_length)

In [ ]:
df_log_item.columns

In [ ]:
df_feats = pd.merge(left = df_train, 
                  right= df_log_item, 
                  how='left',
                  left_on=['user_id'], 
                  right_on=['user_id'])

In [ ]:
df_log_item.isna().sum()

In [ ]:
df_test_feats = pd.merge(left = df_test, 
                  right= df_log_item, 
                  how='left',
                  left_on=['user_id'], 
                  right_on=['user_id'])

In [ ]:
df_feats.columns

In [ ]:
categorical_features = ["app_code", "os_version"] + ['device_type', 'session_id', 'category_1', 'category_2', 'category_3', 'product_type', 'cat1_cat2', 'cat1_cat3', 'cat2_cat3', 'cat1_cat2_cat3']

In [ ]:
Counter(predictions)

In [ ]:
df_test["is_click"] = predictions

In [ ]:
feature_imp = pd.DataFrame(sorted(zip(clf.feature_importance(), 
                                      x_train.columns)), columns=['Value','Feature'])

In [ ]:
feature_imp

In [ ]:
df_test.head()

In [ ]:
sub = df_test[["impression_id", "is_click"]]
sub.to_csv("no_extra_feats_1.csv", index=False)

In [ ]:
df_log_item["category_1"].astype("str") + df_log_item["category_2"].astype("str")